In [2]:
#Librerias
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open
from xml.dom import minidom
from os import listdir 
import re
import numpy as np
import math

In [16]:
#Leer documentos
stop = open("stopWord.txt", "r", encoding="utf-8")
stop_words=stop.readlines()
docs=open("text.txt", "r", encoding="utf-8")
docs=docs.readlines()
steamer=[ ["gato","gata","gatos"], ["pata","patas"],
        ["pato","patos"],["pluma","plumas"], ["ganar","ganan","gana"] ,["garrapata","garrapatas"]]


In [17]:
def remove_StopWords(arrayText):
    newArrayText=[]  
    for txt in arrayText:
        txt=txt+"\n"
        if txt not in stop_words:
            txt=txt.rstrip("\n")
            bol=True
            for arr in steamer:
                if txt in arr:
                    bol=False
                    newArrayText.append(arr[0])
                    break
            if(bol):
                newArrayText.append(txt.rstrip("\n"))              
    return newArrayText

In [18]:
#Procesamiento de un texto utilizando las funciones de Gensim
#Y quitando simbolos
def process(text):     
    stem=re.findall(r'[\w]+', text.lower() ) 
    newStem=remove_StopWords(stem) 
    return newStem

In [41]:
#Crear Diccionario
docDict=[]
for txt in docs:
    docDict.append(process(txt))
dictionary=corpora.Dictionary(docDict) 

In [42]:
#Clase para crear corpus teniendo encuenta el diccionario
class MyCorpus(object):
    def __iter__(self):
        for txt in docs:
            yield dictionary.doc2bow(process(txt))        

In [43]:
corpus_memory=MyCorpus()
#Guardar Corpus
corpora.MmCorpus.serialize('corpus.mm',corpus_memory)
#Cargar Corpus
corpus=corpora.MmCorpus('corpus.mm')
#Crear tfidf del corpus
tfidf=models.TfidfModel(corpus)
#Guardar tfidf
#tfidf.save("tfidfCorpus.mm") 
#Matrix de similidad
index=similarities.MatrixSimilarity(tfidf[corpus]) 

In [47]:
#PUNTO 3 CALCULAR SIMILITUD

QUERIES=["No tengo ninguna palabra en común",
        "La gata y el pato ganan dinero con las garrapatas de las patas y las plumas de la cola",
        "La cucaracha no tiene plumas"]
n=1
arraySortSims=[]
for query in QUERIES:    
    query_doc_bow=dictionary.doc2bow(process(query))
    print(process(query))
    sims=list(index[tfidf[query_doc_bow]])
    listSims=list((enumerate(sims))) 
    sortSims=sorted(listSims,key=lambda pair: pair[1],reverse=True)
    arraySortSims.append(sortSims)
    print("Top 5 documentos de la query",n)
    for dat in sortSims:
        id=dat[0]+1
        print("El documento {} tiene una similitud de {:.4f} ".format(id,dat[1]))
    n+=1
    print("")

['ninguna', 'palabra', 'común']
Top 5 documentos de la query 1
El documento 1 tiene una similitud de 0.0000 
El documento 2 tiene una similitud de 0.0000 
El documento 3 tiene una similitud de 0.0000 
El documento 4 tiene una similitud de 0.0000 
El documento 5 tiene una similitud de 0.0000 

['gato', 'pato', 'ganar', 'dinero', 'garrapata', 'pata', 'pluma', 'cola']
Top 5 documentos de la query 2
El documento 4 tiene una similitud de 0.9252 
El documento 1 tiene una similitud de 0.4221 
El documento 2 tiene una similitud de 0.4221 
El documento 3 tiene una similitud de 0.2477 
El documento 5 tiene una similitud de 0.0559 

['cucaracha', 'pluma']
Top 5 documentos de la query 3
El documento 5 tiene una similitud de 0.8283 
El documento 3 tiene una similitud de 0.2281 
El documento 4 tiene una similitud de 0.1772 
El documento 1 tiene una similitud de 0.0000 
El documento 2 tiene una similitud de 0.0000 



In [54]:
relevanteQ2=[1,1,1,1,1]
relevanteQ1=[0,0,0,0,0]
print(relevanteQ2)
print(relevanteQ1)

[1, 1, 1, 1, 1]
[0, 0, 0, 0, 0]


In [55]:
#Función para llenar la matriz del query con Revelante
# P@K y R@K
def rPK_RK(arrayQuery):
    mQuery=[]
    relevantes=sum(arrayQuery)
    n=0
    for i in arrayQuery: 
        mQuery.append([])
        mQuery[n].append(i)# Relevante
        if(n==0):
            mQuery[n].append(i) #Relevante inicial acumulativo          
        else:
            mQuery[n].append(i+mQuery[n-1][1]) #Relevante acumulativo
        mQuery[n].append( mQuery[n][1]/relevantes ) # R@K
        mQuery[n].append(mQuery[n][1]/(n+1) ) # P@K
        n+=1
    return mQuery

In [59]:
mQuery2=np.array(rPK_RK(relevanteQ2))  
print(mQuery2,"\n")
#mQuery1=np.array(rPK_RK(relevanteQ1))  
#print(mQuery1,"\n")

[[1.  1.  0.2 1. ]
 [1.  2.  0.4 1. ]
 [1.  3.  0.6 1. ]
 [1.  4.  0.8 1. ]
 [1.  5.  1.  1. ]] 



In [65]:
print("Respuestas para el Query 2")
print("P@1= {:.4f} R@1= {:.4f} F@1 {:.4f} ".format(mQuery2[0][3],mQuery2[0][2],(2*mQuery2[0][3]*mQuery2[0][2])/(mQuery2[0][3]+mQuery2[0][2]) ))
print("P@5= {:.4f} R@5= {:.4f} F@5 {:.4f} ".format(mQuery2[4][3],mQuery2[4][2], (2*mQuery2[4][3]*mQuery2[4][2])/(mQuery2[4][3]+mQuery2[4][2])  ))
print("")
print("Respuestas para el Query 1")
print("P@1= {:.4f} R@1= {:.4f} F@1 {:.4f} ".format(0.0,0.0,0.0))
print("P@5= {:.4f} R@5= {:.4f} F@5 {:.4f} ".format(0.0,0.0,0.0))

Respuestas para el Query 2
P@1= 1.0000 R@1= 0.2000 F@1 0.3333 
P@5= 1.0000 R@5= 1.0000 F@5 1.0000 

Respuestas para el Query 1
P@1= 0.0000 R@1= 0.0000 F@1 0.0000 
P@5= 0.0000 R@5= 0.0000 F@5 0.0000 


In [66]:
def dfGainDCGI(query_rel):
    mat=[]
    n=0
    for i in query_rel:
        mat.append([])
        mat[n].append(i) #REl_i
        mat[n].append(1/(math.log(max(n+1,2), 2))) #discount_factor
        mat[n].append(i*mat[n][1])
        if(n==0):
            mat[n].append(mat[n][2]) #DCG_i
        else:
            mat[n].append(mat[n][2]+mat[n-1][3]) #DCG_i
        n+=1
    return mat

In [67]:
def NDCG(matQ,matQN):
    row=len(matQ)
    col=len(matQ[0])
    return matQ[row-1][col-1]/matQN[row-1][col-1] 

In [68]:
escalaR={ 1:4,2:3,3:3,4:5,5:2 } #Query III
vectorEscala=[] 
sortSims=arraySortSims[2]
for a in sortSims:
    id=a[0]+1
    if id in escalaR:
        vectorEscala.append(escalaR[id])
    else:
        vectorEscala.append(0)
vectorEscala

[2, 3, 5, 4, 3]

In [74]:
dcg=np.array(dfGainDCGI( vectorEscala ))
dcgN=np.array(dfGainDCGI( sorted( vectorEscala ,reverse=True)))
#print(dcg,"\n","\n",dcgN)
print("El dcg del query 3 es: ",dcg[4][3])
print("El dcg normalizado del query 3 del query 3 es:",dcgN[4][3])
#ndcg=NDCG(dcg,dcgN) 
#print("NDCG: De los 10 documentos es:",ndcg) 

El dcg del query 3 es:  11.446678442077467
El dcg normalizado del query 3 del query 3 es: 13.254142376861159
